In [31]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import time

In [32]:
import ipynb.fs.full.TrainTest as TrainTest

# nperseg = 2
p = 29 # want this to be high
a = [2]
w = 4  # want this to be low
cw = 9 # want this to be high

# path = './data/created/nperseg' + str(nperseg) + '/'
path = './data/created/nperseg=sqrt/'
file = 'p' + str(p) + '_a' + str(a) + '_w' + str(w) + '_cw' + str(cw)
ext = '.npy'
data = np.load(path + file + ext)

x_train, y_train, x_test, y_test = TrainTest.get_train_test(data, 0.8)

Do RandomizedSearchCV

In [33]:
def getBestParams(clf, param_options):
    clf_grid = GridSearchCV(estimator = clf, param_grid = param_options, cv = 10, verbose=1, n_jobs = -1)
    clf_grid.fit(x_train, y_train)
    return clf_grid.best_params_

In [34]:
from sklearn.metrics import accuracy_score, precision_score, f1_score

# options for nearest neighbors
param_options = {
    'n_neighbors' : range(1, 6),
    'weights' : ['uniform', 'distance'],
    'algorithm' : ['auto'],
    'leaf_size' : [10, 20, 30, 40, 50],
    'p' : [1, 2, 3, 4],
    'metric' : ['euclidean', 'manhattan', 'chebyshev', 'minkowski'],
#     'metric_params' : [],
#     'n_jobs' : []
}

best = getBestParams(KNeighborsClassifier(), param_options)
knn = KNeighborsClassifier(**best)

avg_acc = 0
avg_prec = 0
avg_f1 = 0
avg_train_time = 0
avg_test_time = 0

num = 50
for i in range(num):
    train_start = time.clock()
    knn.fit(x_train, y_train)
    train_end = time.clock()
    
    test_start = time.clock()
    y_pred = knn.predict(x_test)
    test_end = time.clock()
    
    avg_acc += accuracy_score(y_test, y_pred)
    avg_prec += precision_score(y_test, y_pred)
    avg_f1 += f1_score(y_test, y_pred)
    avg_train_time += (train_end - train_start)
    avg_test_time += (test_end - test_start)

avg_acc /= num
avg_prec /= num
avg_f1 /= num
avg_train_time /= num
avg_test_time /= num

with open('./results/knn/' + file + '.txt', 'a+') as f:
    f.write('(nperseg = {}, best = {})\n'.format('sqrt', best))
    f.write('trials         : {}\n'.format(num))
    f.write('avg acc        : {}\n'.format(avg_acc))
    f.write('avg prec       : {}\n'.format(avg_prec))
    f.write('avg f1         : {}\n'.format(avg_f1))
    f.write('avg_train_time : {}\n'.format(avg_train_time))
    f.write('avg_test_time  : {}\n'.format(avg_test_time))
    f.write('-----\n\n')

Fitting 10 folds for each of 800 candidates, totalling 8000 fits


[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 1024 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 2524 tasks      | elapsed:   19.2s
[Parallel(n_jobs=-1)]: Done 4624 tasks      | elapsed:   39.0s
[Parallel(n_jobs=-1)]: Done 6037 tasks      | elapsed:   56.4s
[Parallel(n_jobs=-1)]: Done 7687 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 7985 out of 8000 | elapsed:  1.5min remaining:    0.2s
[Parallel(n_jobs=-1)]: Done 8000 out of 8000 | elapsed:  1.5min finished
